# Libraries

In [1]:
using NCDatasets
using PyPlot
using DataStructures
using Dates

In [2]:
# Directory where you store the seasonal files
wkdir="C:/Users/npages/Desktop/DIVand/resultats/Loire/O2/data_nc/"

"C:/Users/npages/Desktop/DIVand/resultats/Loire/O2/data_nc/"

# Files name

In [3]:
datafile_winter = string(wkdir,"Winter/Water_body_dissolved_oxygen_concentration.4Danl.nc")
datafile_spring = string(wkdir,"Spring/Water_body_dissolved_oxygen_concentration.4Danl.nc")
datafile_summer = string(wkdir,"Summer/Water_body_dissolved_oxygen_concentration.4Danl.nc")
datafile_fall = string(wkdir,"Fall/Water_body_dissolved_oxygen_concentration.4Danl.nc")

datafilemerge = string(wkdir,"output2.nc")



"C:/Users/npages/Desktop/DIVand/resultats/Loire/O2/data_nc/output2.nc"

# Functions definition


    read_obs(datafile)
Read the observations from a netCDF file.
## Example
```julia-repl
julia> obslon, obslat, obsdepth, obstime, obsid = read_obs(datafile)
```


In [4]:
function read_obs(datafile::String)

    NCDatasets.Dataset(datafile, "r") do nc

        obslon = nc["obslon"][:]
        obslat = nc["obslat"][:]
        obsdepth = nc["obsdepth"][:]
        obstime = nc["obstime"][:]
        obsid = nc["obsid"][:]
        
        obslon = convert(Array{Float64,1},obslon)
        obslat = convert(Array{Float64,1},obslat)
        obsdepth = convert(Array{Float64,1},obsdepth)

        return obslon, obslat, obsdepth, obstime, obsid

    end
end

read_obs (generic function with 1 method)


    merge_obsids(obsid1, obsid2)
Merge two arrays of observations
## Example
```julia-repl
julia> obsid = merge_obsids(obsid_old, obsid_new)
```


In [5]:
function merge_obsids(obsid1::Matrix{Char}, obsid2::Matrix{Char},
        obsid3::Matrix{Char}, obsid4::Matrix{Char})
    idlen1, nobs1 = size(obsid1)
    idlen2, nobs2 = size(obsid2)
    idlen3, nobs3 = size(obsid3)
    idlen4, nobs4 = size(obsid4)
    @info(idlen1, idlen2, idlen3, idlen4, nobs1, nobs2, nobs3, nobs4);

    # Allocate new matrix for obsid
    obsid = Array{Char, 2}(undef, maximum((idlen1, idlen2, idlen3, idlen4)),
        nobs1 + nobs2 + nobs3 + nobs4);
    # Merge obsid's
    obsid[1:idlen1, 1:nobs1] = obsid1; # Nolwenn: original code was ... = obsid_old
    obsid[1:idlen2, nobs1 + 1:nobs1 + nobs2] = obsid2; # Nolwenn: original code was ... = obsid_new
    obsid[1:idlen3, nobs1 + nobs2 + 1:nobs1 + nobs2 + nobs3] = obsid3;
    obsid[1:idlen4, nobs1 + nobs2 + nobs3 + 1:nobs1 + nobs2 + nobs3 + nobs4] = obsid4;
    @info(size(obsid));

    return obsid
end

merge_obsids (generic function with 1 method)


    write_obs(datafile)
Write the new observations to a netCDF file from which the variables
corresponding to the observations (obslon, obslat etc) have been removed.
## Example
```julia-repl
julia> write_obs("merged.nc", obslon, obslat, obsdepth, obstime, obsid)
```


In [6]:
function write_obs(datafile::String, obslon::Vector{Float64}, obslat::Vector{Float64},
    obsdepth::Vector{Float64}, obstime::Vector{Dates.DateTime},
    obsid::Matrix{Char})

    idlen, nobs = size(obsid)

    # Write in the new file
    NCDatasets.Dataset(datafilemerge, "a") do nc
        nc.dim["idlen"] = idlen
        nc.dim["observations"] = nobs

        ncobsdepth = defVar(nc,"obsdepth", Float64, ("observations",),
            attrib = OrderedDict(
                        "units" => "meters",
                        "positive" => "down",
                        "long_name" => "depth of the observations",
                        "standard_name" => "depth"
                    )
                )

        ncobsid = defVar(nc,"obsid", Char, ("idlen", "observations"),
            attrib = OrderedDict(
                "long_name" => "observation identifier",
                "coordinates" => "obstime obsdepth obslat obslon",
                )
            )

        ncobslat = defVar(nc,"obslat", Float64, ("observations",),
            attrib = OrderedDict(
                "units" => "degrees_north",
                "long_name" => "latitude of the observations",
                "standard_name" => "latitude"
                )
            )

        ncobslon = defVar(nc,"obslon", Float64, ("observations",),
            attrib = OrderedDict(
                "units" => "degrees_east",
                "long_name" => "longitude of the observations",
                "standard_name" => "longitude"
                )
            )

        ncobstime = defVar(nc,"obstime", Float64, ("observations",),
            attrib = OrderedDict(
                "units" => "days since 1900-01-01 00:00:00",
                "long_name" => "time of the observations",
                "standard_name" => "time"
                )
            )

        ncobslon[:] = obslon
        ncobslat[:] = obslat
        ncobsdepth[:] = obsdepth
        ncobstime[:] = obstime
        ncobsid[:] = obsid;

        return 
    end;
end;

# Main

In [8]:
# Read observations from old and new files
(obslon_winter, obslat_winter, obsdepth_winter,
    obstime_winter, obsid_winter) = read_obs(datafile_winter);
(obslon_spring, obslat_spring, obsdepth_spring,
    obstime_spring, obsid_spring) = read_obs(datafile_spring);
(obslon_summer, obslat_summer, obsdepth_summer,
    obstime_summer, obsid_summer) = read_obs(datafile_summer);
(obslon_fall, obslat_fall, obsdepth_fall,
    obstime_fall, obsid_fall) = read_obs(datafile_fall);

# Append the coordinate arrays
obslon = cat(obslon_winter, obslon_spring, obslon_summer, obslon_fall, dims=1);
obslat = cat(obslat_winter, obslat_spring, obslat_summer, obslat_fall, dims=1);
obsdepth = cat(obsdepth_winter, obsdepth_spring, obsdepth_summer, obsdepth_fall, dims=1);
obstime = cat(obstime_winter, obstime_spring, obstime_summer, obstime_fall, dims=1);

# Merge the obsid's
obsid = merge_obsids(obsid_winter, obsid_spring, obsid_summer, obsid_fall)

# Write the file
write_obs(datafilemerge, obslon, obslat, obsdepth, obstime, obsid)

┌ Info: 27
│   idlen2 = 30
│   idlen3 = 27
│   idlen4 = 24
│   nobs1 = 1935
│   nobs2 = 11661
│   nobs3 = 2759
│   nobs4 = 2107
└ @ Main In[5]:7
┌ Info: (30, 18462)
└ @ Main In[5]:17
